In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedShuffleSplit, train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import RFECV
from scipy.stats import uniform

from utils import impute, plot_scatter_matrix, plot_cross_validation, plot_hyperparameter_tuning

ModuleNotFoundError: No module named 'dotenv'

## 1. Load the Data

In [ ]:
df=pd.read_parquet('../data/wrangled_df.parquet')

## 2. Decision Tree